# PEPSKit

In [5]:
using LinearAlgebra
using Random
using TensorKit
using PEPSKit
using KrylovKit
using OptimKit
using MPSKit

## 1. Basic example 

Simple PEPS optimization example for the spin-$\frac{1}{2}$ square lattice Heisenberg model with

$$
H = \sum_{\langle i,j \rangle} \left(J_x S^x_i S^x_j + J_y S^y_i S^y_j + J_z S^z_i S^z_j\right)
$$

and where we choose $J_x=J_z=-1$ and $J_y=1$. This Hamiltonian can be constructed as a `LocalOperator` in PEPSKit via:

In [6]:
H = square_lattice_heisenberg(Jx = -1, Jy = 1, Jz = -1)

LocalOperator{Tuple{Pair{Tuple{CartesianIndex{2}, CartesianIndex{2}}, TrivialTensorMap{ComplexSpace, 2, 2, Matrix{ComplexF64}}}, Pair{Tuple{CartesianIndex{2}, CartesianIndex{2}}, TrivialTensorMap{ComplexSpace, 2, 2, Matrix{ComplexF64}}}}, ComplexSpace}(ComplexSpace[ℂ^2;;], ((CartesianIndex(1, 1), CartesianIndex(2, 1)) => TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 + 0.0im  -0.25 + 0.0im
, (CartesianIndex(1, 1), CartesianIndex(1, 2)) => TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 

This is a convenient way to store the nearest-neighbour interactions that are themselves stored as `TensorMap`s with structure `(ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)`. Indeed, the physical space is the two-dimensional `ℂ^2`. For the PEPS Ansatz we typically choose a higher dimension for the virtual spaces but for the sake of simplicity we also pick `χbond=2` here so that

In [89]:
χbond = 2
Random.seed!(91283219347)
psi_init = InfinitePEPS(2, χbond)

InfinitePEPS{TrivialTensorMap{ComplexSpace, 1, 4, Matrix{ComplexF64}}}(TrivialTensorMap{ComplexSpace, 1, 4, Matrix{ComplexF64}}[TensorMap(ℂ^2 ← (ℂ^2 ⊗ ℂ^2 ⊗ (ℂ^2)' ⊗ (ℂ^2)')):
[:, :, 1, 1, 1] =
  0.40716886290107507 + 0.5513939535479134im  …  -0.5124646882014363 + 0.4073409890127446im
 0.018434572495667094 - 0.3854096346436205im      1.0483755390301144 + 0.9116887232990566im

[:, :, 2, 1, 1] =
  -0.7906172108704128 + 0.4586324308021981im  …  0.31454771281903543 + 0.016055169067413326im
 -0.17746693324912657 + 0.7052910912220652im      0.6757320297218173 + 0.5641681643777072im

[:, :, 1, 2, 1] =
 -0.015725113902318708 - 0.33214456347145727im  …  -0.31738914776303906 + 0.42521667959690623im
   -1.0306177831069063 - 0.4838625762677157im      0.012572493385981537 - 1.4065039143918943im

[:, :, 2, 2, 1] =
  0.10393519087863327 - 0.9673839431809592im  …    0.3717908137037027 + 0.21934685818702798im
 -0.19670826490575757 + 1.0010041118422948im     -0.04754230579919925 + 0.28599434945998753im


For this random initial PEPS, we first initialize a random CTMRG environment based on the size `χenv = 16` of the corners (and thus chosen equal for all C's)

In [90]:
χenv = 16
env0 = CTMRGEnv(psi_init, ComplexSpace(χenv));

We then optimize this CTMRG environment for the initial PEPS via the CTMRG algorithm with a certain hyperparameter choice and using the `leading_boundary` function:

In [91]:
ctm_alg = CTMRG(;
    tol=1e-10,
    miniter=4,
    maxiter=100,
    verbosity=2,
    svd_alg=SVDAdjoint(; fwd_alg=TensorKit.SVD(), rrule_alg=GMRES(; tol=1e-10)),
    ctmrgscheme=:simultaneous,
)
env_init = leading_boundary(env0, psi_init, ctm_alg);

[ Info: CTMRG init:	obj = -1.789538485284e+00 -4.900827952699e+00im	err = 1.0000e+00
[ Info: CTMRG conv 82:	obj = +7.625582516626e+00	err = 6.0390645562e-11	time = 0.94 sec


As a result the energy density of our initial state is given by

In [92]:
e_init = expectation_value(psi_init, H, env_init)

-0.2346947368092478 + 2.006293619093191e-11im

which is clearly higher than the exact minimum $E_{ex} = -0.6694421$. Applying automatic differentiation (AD) to tis CTMRG based energy evaluation, we obtain a gradient of the energy w.r.t. the PEPS tensor entries and combining both we can perform a gradient descent towards the variational minimum using the `fixedpoint` function

In [93]:
opt_alg = PEPSOptimize(;
    boundary_alg=ctm_alg,
    optimizer=LBFGS(4; maxiter=100, gradtol=1e-3, verbosity=2),
    gradient_alg=LinSolver(; solver=GMRES(; tol=1e-6), iterscheme=:fixed),
    reuse_env=true,
)
result = fixedpoint(psi_init, H, opt_alg, env_init)
result.E

[ Info: CTMRG init:	obj = +7.625582516626e+00	err = 1.0000e+00
[ Info: CTMRG conv 1:	obj = +7.625582516626e+00	err = 2.7488022284e-11	time = 0.01 sec
[ Info: LBFGS: initializing with f = -0.234694736814, ‖∇f‖ = 9.8256e-01
[ Info: CTMRG init:	obj = +1.110332291603e+01 -8.841378857663e-11im	err = 1.0000e+00
[ Info: CTMRG conv 16:	obj = +1.221202493388e+01	err = 4.1788461434e-11	time = 0.15 sec
[ Info: LBFGS: iter    1: f = -0.476599368773, ‖∇f‖ = 2.2341e-01, α = 1.00e+00, m = 0, nfg = 1
[ Info: CTMRG init:	obj = +1.250859928053e+01	err = 1.0000e+00
[ Info: CTMRG conv 14:	obj = +1.251193059064e+01	err = 3.7822657201e-11	time = 0.16 sec
[ Info: LBFGS: iter    2: f = -0.512844811020, ‖∇f‖ = 1.1340e-01, α = 1.00e+00, m = 1, nfg = 1
[ Info: CTMRG init:	obj = +1.268953180948e+01	err = 1.0000e+00
[ Info: CTMRG conv 14:	obj = +1.269154795369e+01	err = 6.4148855847e-11	time = 0.20 sec
[ Info: LBFGS: iter    3: f = -0.537558530180, ‖∇f‖ = 9.3445e-02, α = 1.00e+00, m = 2, nfg = 1
[ Info: CTMRG init

-0.6624806307242881

already approaching the exact minimum quite ok. 

## 2. LocalOperator

### 2.1. Heisenberg (again)

Our first step consisted of constructing the Hamiltonian. Some particular models are already in `models.jl` but generally one builds a local Hamitonian $H$ by defining a `lattice`, i.e. a unit cell of physical spaces, and `terms`, corresponding to the terms contributing to $H$. For the Heisenberg model, the physical space is the same on every site and given by

In [94]:
physical_space = ComplexSpace(2)

ℂ^2

so that

In [95]:
lattice = fill(physical_space, 1, 1)

1×1 Matrix{ComplexSpace}:
 ℂ^2

The Hamiltonian consists of identical horizontal and vertical nearest-neighbour terms that we can construct as

In [96]:
Jx = -1
Jy = 1
Jz = -1

T = ComplexF64
σx = TensorMap(T[0 1; 1 0], physical_space, physical_space)
σy = TensorMap(T[0 im; -im 0], physical_space, physical_space)
σz = TensorMap(T[1 0; 0 -1], physical_space, physical_space)

h = 1/4*((Jx * σx ⊗ σx) + (Jy * σy ⊗ σy) + (Jz * σz ⊗ σz))

TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 + 0.0im  -0.25 + 0.0im


These are exactly the `TensorMap`s present in the `H` we constructed via the direct command before. We can again put them in a `LocalOperator` by turning them into `terms`: i.e. a tuple of the sites where `h` acts (labeled as a matrix) and their corresponding Hamiltonian contribution:

In [97]:
term1 = (CartesianIndex(1, 1), CartesianIndex(1, 2)) => h
term2 = (CartesianIndex(1, 1), CartesianIndex(2, 1)) => h

(CartesianIndex(1, 1), CartesianIndex(2, 1)) => TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 + 0.0im  -0.25 + 0.0im


The `LocalOperator` is then given by

In [98]:
HH = LocalOperator(lattice, term1, term2)

LocalOperator{Tuple{Pair{Tuple{CartesianIndex{2}, CartesianIndex{2}}, TrivialTensorMap{ComplexSpace, 2, 2, Matrix{ComplexF64}}}, Pair{Tuple{CartesianIndex{2}, CartesianIndex{2}}, TrivialTensorMap{ComplexSpace, 2, 2, Matrix{ComplexF64}}}}, ComplexSpace}(ComplexSpace[ℂ^2;;], ((CartesianIndex(1, 1), CartesianIndex(1, 2)) => TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 + 0.0im  -0.25 + 0.0im
, (CartesianIndex(1, 1), CartesianIndex(2, 1)) => TensorMap((ℂ^2 ⊗ ℂ^2) ← (ℂ^2 ⊗ ℂ^2)):
[:, :, 1, 1] =
 -0.25 + 0.0im   0.0 + 0.0im
   0.0 + 0.0im  -0.5 + 0.0im

[:, :, 2, 1] =
  0.0 + 0.0im  0.0 + 0.0im
 0.25 + 0.0im  0.0 + 0.0im

[:, :, 1, 2] =
 0.0 + 0.0im  0.25 + 0.0im
 0.0 + 0.0im   0.0 + 0.0im

[:, :, 2, 2] =
 -0.5 + 0.0im    0.0 + 0.0im
  0.0 

This is exactly the same H as the one constructed via the direct command:

In [99]:
HH-H

LocalOperator{Tuple{}, ComplexSpace}(ComplexSpace[ℂ^2;;], ())

For Hamiltonians solely consisting out of identical nearest-neighbour contributions, one can also use

In [100]:
HHH = PEPSKit.nearest_neighbour_hamiltonian(lattice, h)
H-HHH

LocalOperator{Tuple{}, ComplexSpace}(ComplexSpace[ℂ^2;;], ())

### 2.2 p-wave superconductor

As a slightly more involved example, we consider the fermionic p-wave superconductor with Hamiltonian:

$$
H = - t \sum_\mathbf{n} \left( c_\mathbf{n}^\dagger c_{\mathbf{n}_\rightarrow}+ c_\mathbf{n}^\dagger c_{\mathbf{n}_\uparrow} +h.c.\right) -\mu \sum_\mathbf{n} c_\mathbf{n}^\dagger c_{\mathbf{n}} - \Delta \sum_\mathbf{n} \left(c_\mathbf{n}^\dagger c_{\mathbf{n}_\rightarrow}^\dagger + i c_\mathbf{n}^\dagger c_{\mathbf{n}_\uparrow}^\dagger + h.c.\right)
$$

consisting out of identical horizontal and vertical nearest-neighbour hopping terms, an on-site chemical potential term, and nearest-neighbour pairing terms, equal in strength but with a different phase in the horizontal and vertical direction. We choose:

In [101]:
t = 1
μ = 2
Δ = 1;

The physical space is again two-dimensional but fermionic now. This implies that it decomposes in an even and odd fermion parity sector, i.e. it is a $\mathbb{Z}_2$-graded Hilbert space. To work with fermionic tensors consisting out of spaces exhibiting this $f\mathbb{Z}_2$ symmetry, we define

In [102]:
physical_space = Vect[FermionParity](0 => 1, 1 => 1)

Vect[FermionParity](0=>1, 1=>1)

Using these spaces in combination with `TensorMap`, the on-site contribution to $H$ can for instance be defined as:

In [103]:
h0 = TensorMap(zeros, T, physical_space ← physical_space)
block(h0, FermionParity(1)) .= -μ
h0

TensorMap(Vect[FermionParity](0=>1, 1=>1) ← Vect[FermionParity](0=>1, 1=>1)):
* Data for sector (FermionParity(0),) ← (FermionParity(0),):
 0.0 + 0.0im
* Data for sector (FermionParity(1),) ← (FermionParity(1),):
 -2.0 + 0.0im


Working out the horizontal and vertical terms, one obtains:

In [104]:
hx = TensorMap(zeros, T, physical_space^2 ← physical_space^2)
block(hx, FermionParity(0)) .= [0 -Δ; -Δ 0]
block(hx, FermionParity(1)) .= [0 -t; -t 0]
hx;

In [105]:
hy = TensorMap(zeros, T, physical_space^2 ← physical_space^2)
block(hy, FermionParity(0)) .= [0 Δ*im; -Δ*im 0]
block(hy, FermionParity(1)) .= [0 -t; -t 0]
hy;

All of these contributions can then be combined in a `LocalOperator` by first defining the `lattice` with again `physical_space` at each site and thus:

In [106]:
lattice = fill(physical_space, 1, 1)

1×1 Matrix{GradedSpace{FermionParity, Tuple{Int64, Int64}}}:
 Vect[FermionParity](0=>1, 1=>1)

For the `terms` we have:

In [107]:
term0 = (CartesianIndex(1, 1),) => h0
termx = (CartesianIndex(1, 1), CartesianIndex(1, 2)) => hx
termy = (CartesianIndex(1, 1), CartesianIndex(2, 1)) => hy;

so that

In [108]:
H = LocalOperator(lattice, term0, termx, termy);

The p-wave superconductor is also directly implemented in `models.jl` and we can check:

In [109]:
H - square_lattice_pwave(t=1, μ=2, Δ=1)

LocalOperator{Tuple{}, GradedSpace{FermionParity, Tuple{Int64, Int64}}}(GradedSpace{FermionParity, Tuple{Int64, Int64}}[Vect[FermionParity](0=>1, 1=>1);;], ())

### 2.3 t-J model for nickelates (Gleb)

More complexity can be added. For instance, one can opt for a larger unit cell with certain different physical spaces to target a specific filling fraction. In specifying the Hamiltonian, one then has to add all `terms` in the same way as before but now for the larger unit cell. Example of code Gleb uses within the nickelates project where all of the tensors incorporate $f\mathbb{Z}_2$ symmetry as well as $U(1)$ and $SU(2)$:


```
V_phys_A = Vect[FermionParity ⊠ U1Irrep ⊠ SU2Irrep]((1,-1,1/2) => D_phys, (0,0,0) => D_phys);
V_phys_B = Vect[FermionParity ⊠ U1Irrep ⊠ SU2Irrep]((1,1,1/2) => D_phys, (0,0,0) => D_phys);

lattice = [V_phys_A V_phys_B; V_phys_B V_phys_A];

H = LocalOperator(lattice,
(CartesianIndex(1,1), CartesianIndex(1,2)) => two_site_AB,
(CartesianIndex(1,1), CartesianIndex(2,2)) => two_site_AA,
(CartesianIndex(2,2), CartesianIndex(3,3)) => two_site_AA,
(CartesianIndex(2,1), CartesianIndex(1,2)) => two_site_BB,
(CartesianIndex(1,2), CartesianIndex(0,3)) => two_site_BB,
(CartesianIndex(2,1), CartesianIndex(3,2)) => two_site_BB,
(CartesianIndex(2,2), CartesianIndex(3,1)) => two_site_AA,
(CartesianIndex(2,2), CartesianIndex(2,3)) => two_site_AB,
(CartesianIndex(2,2), CartesianIndex(1,3)) => two_site_AA,
(CartesianIndex(1,2), CartesianIndex(2,3)) => two_site_BB
);
```

## 3. Algorithms

For PEPS optimization we require: 

- a `boundary_alg` to approximate the evaluation of expectation values and taking into account the infinite nature of the PEPS (typically we use CTMRG)
- a `gradient_alg` determining how exactly the energy gradient is calculated by applying AD to the `boundary_alg`
- an `opt_alg` determining how these are combined to decrease the energy towards its variational optimum

Maybe we can talk about:

- CTMRG for fermions
- fixed-point calculation of gradient and related issues
- OptimKit issues
- VUMPS as an alternative `boundary_alg`

## 4. Initial guesses and Environments

### 4.1. p-wave superconductor (again)

We already constructed the `LocalOperator` for the p-wave superconductor model and already know what the physical space looks like.

In [110]:
physical_space = Vect[FermionParity](0 => 1, 1 => 1)
H = square_lattice_pwave(t=1, μ=2, Δ=1);

For the initial guess of the PEPS state we use similar virtual fermionic spaces with both even and odd sectors and with half the PEPS bond dimension $D$ as their respective size.

In [111]:
D = 2
virtual_space = Vect[FermionParity](0 => D/2, 1 => D/2);

The initial guess for the PEPS state can then easily be constructed by

In [112]:
psi_init = InfinitePEPS(physical_space, virtual_space);

Similarly, we define the environment space for the CTMRG algorithm in the most simple way by allowing both sectors and making them equally large:

In [113]:
χ = 16
env_space = Vect[FermionParity](0 => χ/2, 1 => χ/2);

The environments can then be initialized via

In [114]:
env0 = CTMRGEnv(psi_init, env_space);

Running CTMRG for `psi_init`, one obtains

In [115]:
ctm_alg = CTMRG(; tol=1e-8, maxiter=200, verbosity=2, ctmrgscheme=:sequential)
env_init = leading_boundary(env0, psi_init, ctm_alg);

[ Info: CTMRG init:	obj = -4.859764831139e-01 -4.885414980757e-01im	err = 1.0000e+00
[ Info: CTMRG conv 24:	obj = +2.576094821044e+00 -1.624377833335e-11im	err = 6.2854458796e-09	time = 0.60 sec


so that the PEPS optimization can be run via

In [116]:
opt_alg = PEPSOptimize(;
    boundary_alg=ctm_alg,
    optimizer=LBFGS(4; maxiter=10, gradtol=1e-3, verbosity=2),
    gradient_alg=LinSolver(; solver=GMRES(; tol=1e-3, maxiter=2), iterscheme=:diffgauge),
    reuse_env=true,
)
result = fixedpoint(psi_init, H, opt_alg, env_init)
result.E

[ Info: CTMRG init:	obj = +2.576094821044e+00 -1.624377833335e-11im	err = 1.0000e+00
[ Info: CTMRG conv 1:	obj = +2.576094821017e+00	err = 2.6736402378e-09	time = 0.01 sec
[ Info: LBFGS: initializing with f = -1.237653160683, ‖∇f‖ = 7.7779e-01
[ Info: CTMRG init:	obj = +3.207665202537e+00 -4.184797091615e-12im	err = 1.0000e+00
[ Info: CTMRG conv 76:	obj = +2.861133300342e+00	err = 8.7081710124e-09	time = 0.85 sec
[ Info: LBFGS: iter    1: f = -1.845240766075, ‖∇f‖ = 6.2280e-01, α = 1.00e+00, m = 0, nfg = 1
[ Info: CTMRG init:	obj = +5.855782623297e+00	err = 1.0000e+00
┌ Warning: CTMRG cancel 200:	obj = +2.776971315118e+00	err = 1.4021494878e-07	time = 2.18 sec
└ @ PEPSKit ~/Documents/project_gluekit/dev/PEPSKit/src/algorithms/ctmrg/ctmrg.jl:167
[ Info: LBFGS: iter    2: f = -2.064870424117, ‖∇f‖ = 3.3021e-01, α = 1.00e+00, m = 1, nfg = 1
[ Info: CTMRG init:	obj = +3.019365748282e+00	err = 1.0000e+00
┌ Warning: CTMRG cancel 200:	obj = +3.477131542267e+00	err = 7.5559673141e-06	time = 2.

-2.495782877411683

approximating the exact energy density $E_{ex}=-2.6241$.

Sometimes it helps to allow for a larger unit cell even though the Hamiltonian does not necessarily seem to require this (e.g. to allow for some symmetry breaking or to yield a more stable convergence). We therefore randomly initialize the PEPS again but now with a 2x2 unit cell

In [118]:
expectation_value(result.peps, H, result.env)

-2.495782877411683 - 6.808044186643143e-17im

In [119]:
unitcell = (2, 2)
psi_init = InfinitePEPS(physical_space, virtual_space; unitcell);
norm(psi_init[1,1]-psi_init[2,2])

5.970056574194527

The `LocalOperator` then requires the same unit cell but consists of identical terms on each site. We can simply construct this repeated operator via

In [120]:
H = square_lattice_pwave(t=1, μ=2, Δ=1; unitcell);

Again using the same environment space for all CTMRG tensors, we initialize the environments as

In [121]:
env0 = CTMRGEnv(psi_init, env_space);

Running CTMRG for the initial guess then yields

In [122]:
ctm_alg = CTMRG(; tol=1e-8, maxiter=200, verbosity=2, ctmrgscheme=:sequential)
env_init = leading_boundary(env0, psi_init, ctm_alg);

[ Info: CTMRG init:	obj = +1.497060085161e+03 -3.660993347859e+03im	err = 1.0000e+00
[ Info: CTMRG conv 26:	obj = +1.064720439459e+02 +2.208523231815e-10im	err = 4.4347739049e-09	time = 1.15 sec


while PEPS optimization results in

In [123]:
opt_alg = PEPSOptimize(;
    boundary_alg=ctm_alg,
    optimizer=LBFGS(4; maxiter=10, gradtol=1e-3, verbosity=2),
    gradient_alg=LinSolver(; solver=GMRES(; tol=1e-3, maxiter=2), iterscheme=:diffgauge),
    reuse_env=true,
)
result = fixedpoint(psi_init, H, opt_alg, env_init)
result.E / prod(size(psi_init))

[ Info: CTMRG init:	obj = +1.064720439459e+02 +2.208523231815e-10im	err = 1.0000e+00
[ Info: CTMRG conv 1:	obj = +1.064720439419e+02 -1.636888571076e-10im	err = 1.1982126663e-09	time = 0.08 sec
[ Info: LBFGS: initializing with f = -4.344228194112, ‖∇f‖ = 2.3273e+00
[ Info: CTMRG init:	obj = +1.527660060610e+02 -5.113623502939e-10im	err = 1.0000e+00
[ Info: CTMRG conv 17:	obj = +1.718535869293e+02	err = 3.2719111316e-09	time = 0.72 sec
[ Info: LBFGS: iter    1: f = -6.373388639462, ‖∇f‖ = 1.8881e+00, α = 1.00e+00, m = 0, nfg = 1
[ Info: CTMRG init:	obj = +1.752490373850e+03	err = 1.0000e+00
[ Info: CTMRG conv 16:	obj = +2.300806548110e+03	err = 5.3141434619e-09	time = 0.68 sec
[ Info: LBFGS: iter    2: f = -8.752468173795, ‖∇f‖ = 8.6981e-01, α = 1.00e+00, m = 1, nfg = 1
[ Info: CTMRG init:	obj = +5.365743935627e+03	err = 1.0000e+00
[ Info: CTMRG conv 15:	obj = +5.530650722916e+03	err = 2.2771038869e-09	time = 0.62 sec
[ Info: LBFGS: iter    3: f = -9.260603748563, ‖∇f‖ = 4.9148e-01, α =

-2.602454779407535

### 4.2. More involved symmetries

Consider a spinful fermionic hopping model with 

$$
H = - t \sum_{\sigma,\mathbf{n}} \left( c_{\sigma,\mathbf{n}}^\dagger c_{\sigma,\mathbf{n}_\rightarrow}+ c_{\sigma,\mathbf{n}}^\dagger c_{\sigma,\mathbf{n}_\uparrow} +h.c.\right)
$$

and $\sigma \in \{\uparrow,\downarrow\}$, having $f\mathbb{Z}_2$, $U(1)$ and $SU(2)$ symmetry. Allowing for double occupancy in this case, the physical space can be defined as

In [51]:
physical_space = Vect[FermionParity ⊠ U1Irrep ⊠ SU2Irrep]((0,0,0) => 1, (1,1,1/2) => 1, (0,2,0) => 1)

Vect[(FermionParity ⊠ Irrep[U₁] ⊠ Irrep[SU₂])]((0, 0, 0)=>1, (0, 2, 0)=>1, (1, 1, 1/2)=>1)

Note that the virtual PEPS legs will also have this symmetry. However, they can realize a larger degeneracy within these sectors but also have symmetry sectors with higher spin labels, e.g. $(1,3,3/2)$. As one has to restrict to a finite number of sectors with reasonable degeneracies to not make the bond dimension too large, it is a priori not clear which spin labels to consider and with what degeneracy.

This problem also appears in the environment spaces. Utilizing CTMRG, one could perform the necessary truncations based on a certain Schmidt cut and select/add spaces based on this. However, the initial and final environment spaces of a CTMRG could differ then, invalidating the use of fixed point calculations for the energy gradient via AD.